In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


<div>
<img src="../images/CNN.jpeg" width="500"/>
</div>

In [7]:

class DatasetCIFAR10(Dataset):
    def __init__(self, train=True, transform=None):
        self.dataset = CIFAR10(root='../data', train=train, download=False, transform=transform)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return image, label


In [8]:


# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load data
train_dataset = DatasetCIFAR10(train=True, transform=transform)
test_dataset = DatasetCIFAR10(train=False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



RuntimeError: Dataset not found or corrupted. You can use download=True to download it

In [4]:
cifar10_classes = {
    0: "Airplane",
    1: "Automobile",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Ship",
    9: "Truck"
}

In [5]:
train_dataset.__getitem__(5)

NameError: name 'train_dataset' is not defined

In [ ]:


def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
imshow(train_dataset.__getitem__(5)[0])

In [ ]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.dropout(x) #NOTE: MODEL.EVAL() REALLY REALLY MATTERS NOW!!!
        x = self.fc2(x)
        return x

In [ ]:
model = SimpleCNN()

red_car = train_dataset.__getitem__(5)[0]

print(model.conv1(red_car),model.conv1(red_car).size())

In [ ]:
'''
Maybe for your version?
if torch.cuda.is_available():
    device = torch.device('cuda')  # Use GPU
else:
    device = torch.device('cpu')  
'''


# Initialize the network, loss function, and optimizer
model = SimpleCNN()

device = torch.device('mps')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


In [ ]:

device = torch.device('cpu')
model.to(device)

# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the test images: {100 * correct / total} %")


### Exercises

Do the work necessary to create a nice confusion matrix - including class labels!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
import torchvision.transforms as transforms

# Load LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
images = lfw_people.images
labels = lfw_people.target
label_names = lfw_people.target_names

print(f"Number of images: {images.shape[0]}")
print(f"Image shape: {images.shape[1:]}")


# Visualize a few examples
fig, axes = plt.subplots(1, 5, figsize=(15, 8))
for i in range(5):
    axes[i].imshow(images[i], cmap='gray')
    axes[i].set_title(label_names[labels[i]])
    axes[i].axis('off')
plt.show()


# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
